In [27]:

import numpy as np
import pandas as pd
from collections import Counter
import re
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics
from sklearn.model_selection import train_test_split

df_train = pd.read_csv(r'C:\Users\user\Desktop\RF_project\data_1\main_task.csv')
df_test = pd.read_csv(r'C:\Users\user\Desktop\RF_project\data_1\kaggle_task.csv')
sample_submission = pd.read_csv(r'C:\Users\user\Desktop\RF_project\data_1\sample_submission.csv')

df_test['sample'] = 0
df_train['sample'] = 1
df_test['Rating'] = 0 # в тесте нет Rating, поэтому пока проставим нулями
data_1 = df_test.append(df_train, sort=False).reset_index(drop=True)


def preprocession_data(df_input):
	
	"""Includes several fucntion to pre-process the row data"""
	
	df_output = df_input.copy()
	
	# 1. Просмотр данных
	print("Количество пропусков\n", df_output.isna().sum())
	print("Количество уникальных значений\n", df_output.nunique())
	print("Количество уникальных значений в тестовой выборке\n", df_output.nunique())
	sns.heatmap(df_output.isnull(), cmap = 'summer')
	plt.title('Диаграмма 1 Пропущенные значения по фичам')
	
	# 2. обработка Nan и пропущенных значений
	
	df_output['Cuisine Style'].fillna("['Others']", inplace=True)
	df_output['Price Range'].fillna("['uncertain']", inplace=True)
	df_output.Reviews = df_output.Reviews.fillna('[[], []]', inplace=True)
	df_output['Number of Reviews'].fillna(df_output['Number of Reviews'].mean(), inplace = True)

	# 3. Encoding - создаем dummies для городов и ценовой категории
	df_output = pd.get_dummies(df_output, columns = ['City'], dummy_na=True)
	df_output = pd.get_dummies(df_output, columns = ['Price Range'], dummy_na=True)
	

	# 4. Генерация новых фитчей
	# количество ресторанов в городе
	city_count_dic = dict(df_output['City'].value_counts())
	df_output['main_rest'] = df_output.City.map(city_count_dic)
	# виды кухонь
	df_output['Cuisine Style list'] = df_output['Cuisine Style'].str.findall(r"'(\b.*?\b)'")
	# относительный рейтинг
	df_output['Relative_ranking'] = df_output['Ranking']/df_output['main_rest']
	# количество отзвов в городе в городе
	reviews_city_dict = dict(df_output.groupby('City')['Number of Reviews'].sum())
	df_output['num_reviews_city'] = df_output.City.map(reviews_city_dict)
	df_output['relative_rank_reviews'] = df_output['Ranking']/df_output['num_reviews_city']

	# дамми переменная для стилей кухни
	cuisine_stule_dummy = df_output['Cuisine Style list'].apply(lambda x: pd.Series([1]*len(x)))
	df_output = pd.merge(df_output, cuisine_stule_dummy, left_index = True, raight_index = True, how = 'left')

	# 5. Удаляем признаки с object, которые не успели обработать
	# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
	object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
	df_output.drop(object_columns, axis = 1, inplace=True)
	return df_output


def ML_proc(input_data):

	output_data = input_data.copy()
	train_data = output_data.query('sample == 1').drop(['sample'], axis=1)
	test_data = output_data.query('sample == 0').drop(['sample'], axis=1)
	y = train_data.Rating.values            # наш тарге
	X = train_data.drop(['Rating'], axis=1)
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)
	model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=47)
	model.fit(X_train, y_train)

	y_pred = model.predict(X_test)
	print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

	return y_pred


C:\Users\user\AppData\Local\Temp/ipykernel_9496/1924639656.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_1 = df_test.append(df_train, sort=False).reset_index(drop=True)
